In [1]:
import sys
import numpy as np
import tqdm

PI = np.pi
SIN = np.sqrt(3)/2.0
CONST1 = 2/np.sqrt(6)
CONST2 = 1/np.sqrt(2)

#### 逆格子空間での射影

In [2]:
def prjctn_recipro_par(r,a=1.0,c=1.0):
    """
    parallel component of 5D reciprocal lattice vector
    input
    list:r, 5D reflection index
    float:a, lattice constant
    float:c, lattice constant
    """
    x = (1*r[0]+SIN*r[1]+0.5*r[2]       )/a*CONST2
    y = (       0.5*r[1]+SIN*r[2]+1*r[3])/a*CONST2
    z = r[4]/c
    return [x,y,z]

def prjctn_recipro_perp(r,a=1.0):
    """
    perpendicular component of 5D reciprocal lattice vector
    input
    list:r, 4D reflection index
    float:a, lattice constant
    """
    x=(1*r[0]-SIN*r[1]+0.5*r[2]     )/a*CONST2
    y=(       0.5*r[1]-SIN*r[2]+r[3])/a*CONST2
    return [x,y]


In [3]:
def prjctn_recipro_par_length(r,a=1.0,c=1.0):
    [x,y,z]=prjctn_recipro_par(r,a,c)
    return np.sqrt(x**2+y**2+z**2)

def prjctn_recipro_perp_length(r,a=1.0):
    [x,y]=prjctn_recipro_perp(r,a)
    return np.sqrt(x**2+y**2)


#### 実空間での射影

In [4]:
def prjctn_direct_par(r,a=1.0,c=1.0):
    """
    parallel component of 5D reciprocal lattice vector
    input
    list:r, 5D reflection index
    float:a, lattice constant
    float:c, lattice constant
    """
    x = ( SIN*r[0]+r[1]     -0.5*r[3])*a*CONST1
    y = (-0.5*r[0]     +r[2]+SIN*r[3])*a*CONST1
    z = r[4]*c
    return [x,y,z]

def prjctn_direct_perp(r,a=1.0):
    """
    perpendicular component of 5D reciprocal lattice vector
    input
    list:r, 5D reflection index
    float:a, lattice constant
    """
    x = (-SIN*r[0]+r[1]-0.5*r[3])*a*CONST1
    y = (-0.5*r[0]+r[2]-SIN*r[3])*a*CONST1
    return [x,y]


In [5]:
def prjctn_direct_par_length(r,a=1.0,c=1.0):
    [x,y,z]=prjctn_direct_par(r,a,c)
    return np.sqrt(x**2+y**2+z**2)

def prjctn_direct_perp_length(r,a=1.0):
    [x,y]=prjctn_direct_perp(r,a)
    return np.sqrt(x**2+y**2)


#### フーリエ変換

In [6]:
import math
from scipy import integrate
import cmath

In [7]:
PI = np.pi
TWOPI = 2.0*PI

#### 指数発生

In [8]:
PI=np.pi
CONST3=np.cos(PI/12)
CONST4=np.sin(PI/12)
EPS=1E-6
TAU = (1+np.sqrt(5))/2

In [9]:
def asymmetric(h):
    """
    # 非対称領域の内外判定
    # dodecagonal lattice
    """
    [x,y,z] = prjctn_recipro_par(h)
    vec = np.array([x,y,0])
    
    tmp = np.array([1,0,0])
    if np.cross(tmp,vec)[2] > -EPS:
        tmp = np.array([CONST3,CONST4,0])
        if np.cross(tmp,vec)[2] < EPS:
            if z>=0.0:
                return True
            else:
                return False
        else:
            return False
    else:
        return False

In [10]:
def mattrix_dodeca_sym():
    """
    dodecagonal symmetry operations
    """
    #c12
    m1=np.array([[ 0, 0, 0,-1, 0],\
                [ 1, 0, 0, 0, 0],\
                [ 0, 1, 0, 1, 0],\
                [ 0, 0, 1, 0, 0],\
                [ 0, 0, 0, 0, 1]])
    # mirror
    m2=np.array([[ 0, 0, 1, 0, 0],\
                [ 0, 1, 0, 0, 0],\
                [ 1, 0, 0, 0, 0],\
                [ 0, 1, 0,-1, 0],\
                [ 0, 0, 0, 0, 1]])
    # mirror`
    m3=np.array([[0, 0, 0, 1, 0],\
                [ 0, 0, 1, 0, 0],\
                [ 0, 1, 0, 0, 0],\
                [ 1, 0, 0, 0, 0],\
                [ 0, 0, 0, 0, 1]])
    # inversion
    m4=np.array([[-1, 0, 0, 0, 0],\
                [ 0,-1, 0, 0, 0],\
                [ 0, 0,-1, 0, 0],\
                [ 0, 0, 0,-1, 0],\
                [ 0, 0, 0, 0,-1]])
    # -12
    m5=m4@m1
    return m1,m2,m3,m4,m5

In [11]:
def dodecasymop():
    """
    decagonal symmetry operations
    """
    m1,m2,m3,m4,m5 = mattrix_dodeca_sym()
    
    symop=[]
    """
    for k in range(2):
        for j in range(2):
            for i in range(12):
                s1=matrix_pow(m1,i) # c12
                s2=matrix_pow(m3,j) # mirror, sigma
                s3=matrix_pow(m4,k) # mirror, sigma`
                tmp=np.dot(s2,s3)
                tmp=np.dot(tmp,s1)
                symop.append(tmp)
    """

    for k in range(2):
        for j in range(2):
            for i in range(12):
                s1=matrix_pow(m1,i) # c12
                s2=matrix_pow(m3,j) # mirror, sigma
                s3=matrix_pow(m4,k) # mirror, sigma`
                tmp=np.dot(s2,s3)
                tmp=np.dot(tmp,s1)
                symop.append(tmp)
    
    # for j in range(2):
    #     for i in range(12):
    #         s1=matrix_pow(m5,i) # -12
    #         s2=matrix_pow(m2,j) # mirror y
    #         tmp=np.dot(s1,s2)
    #         symop.append(tmp)
    return symop

In [12]:
def equivalent_sym_op(h):
    """
    hと等価な反射指数を作る対称操作インデックスを求める。
    """
    a=dodecasymop()
    lst=[]
    for i1,b in enumerate(a):
        h1=np.dot(b,h)
        if i1==0:
            lst.append(i1)
        else:
            flg=0
            for i2 in lst:
                h2=np.dot(a[i2],h)
                if np.all(h1==h2):
                    flg+=1
                    break
                else:
                    pass
            if flg==0:
                lst.append(i1)
    return lst

In [13]:
def matrix_pow(array_1, n):
    mx = array_1.shape[0]
    my = array_1.shape[1]
    array_2 = np.identity(mx, dtype=int)
    if mx == my:
        if n == 0:
            return np.identity(mx, dtype=int)
        elif n<0:
            return np.zeros((mx,mx), dtype=int)
        else:
            for i in range(n):
                array_2 = np.dot(array_2,array_1)
            return array_2
    else:
        print('ERROR: matrix has not regular shape')
        return np.array([0])

In [14]:
def gen_ref(hmax,lmax,a_max,c_max,qemax,qimax,flag):
    def reflection_condition(n1,n2,n3,n4,n5):
        """
        set your condition here.
        """
        if n1==0 and n2==0 and n3==0 and n4==0 and n5==0:
            return False
        else:
            return True
        # if n5%2!=0 and n1==0 and n2==0 and n3==0 and n4==0:
        #     return False
        # elif n5%2!=0 and n1+n2+n3 == 2*n4:
        #     return True
        # else:
        #     return True

    ref_dict={}  # 5D indices
    for h1 in tqdm.tqdm(range(-hmax, hmax+1), desc='generate ref'):
        for h2 in range(-hmax, hmax+1):
            for h3 in range(-hmax, hmax+1):
                for h4 in range(-hmax, hmax+1):
                    for h5 in range(-lmax, lmax+1):           
                        if reflection_condition(h1,h2,h3,h4,h5):
                            hkl=[h1,h2,h3,h4,h5]
                            if flag=='unique':
                                if asymmetric(hkl): # asymmetric unit
                                    q_par = prjctn_recipro_par_length(hkl,a_max,c_max)
                                    q_perp = prjctn_recipro_perp_length(hkl,a_max)
                                    if q_par<qemax and q_perp<qimax:
                                        indice_str='_'.join(map(str,map(int,hkl)))
                                        ref_dict[indice_str]=[np.array(hkl),1,q_par,q_perp]
                                    else:
                                        pass
                                else:
                                    pass
                            else:
                                q_par = prjctn_recipro_par_length(hkl,a_max,c_max)
                                q_perp = prjctn_recipro_perp_length(hkl,a_max)
                                if q_par<qemax and q_perp<qimax:
                                    indice_str='_'.join(map(str,map(int,hkl)))
                                    ref_dict[indice_str]=[np.array(hkl),1,q_par,q_perp]
                                else:
                                    pass
                        else:
                            pass
    for indice_str in tqdm.tqdm(ref_dict, desc='multiplicity'):
        a=equivalent_sym_op(ref_dict[indice_str][0])
        ref_dict[indice_str][1]=int(len(a))
    return ref_dict

In [15]:
# a=equivalent_sym_op(np.array([1,2,3,4,5]))

In [16]:
# print(len(a))

In [17]:
hmax=5
lmax=4
# a=7.886
# c=10.403
# a = 9.046
# c = 10.397
# a = 7
# c = 12
a_max=9
c_max=12

wvl_dic={} #X線の波長
wvl_dic['Cu_Ka'] = 1.54059 # in Ang.
wvl_dic['Mo_Ka'] = 0.71069 # in Ang.
wvl = wvl_dic['Cu_Ka']
element = 'Ta'

qemax = 2/wvl # in Ang.^{-1}
qimax = 0.5

flag = 'unique'

#五次元構造の情報
#五次元格子の各頂点に占有領域Aがある
position = np.array([0,0,0,0,0]) #占有領域od_Bの中心位置
#占有領域Aの定義
shape = 'triangle'

site_symmetry = [-1]



In [18]:
peaks_dict = gen_ref(hmax,lmax,a_max,c_max,qemax,qimax,flag)

multiplicity: 100%|████████████████████████████████████████████████████████████████████| 1934/1934 [00:06<00:00, 288.93it/s]


In [19]:
def reflection_list(filename,hmax,lmax,qemax,qimax,a=1.0,c=1.0,flag='unique'):
    """ 
    make a reflection list file.
    this generates reflection indices list
    :param string basename:
    :param int hmax:
    :param float qimax:
    :param string flag: ['unique' or 'u' or 1] or 'all'
    """
    ref_dict = peaks_dict
    f = open('%s'%(filename),'w', encoding="utf-8", errors="ignore")
    f.write('%d %4.2f %4.2f %4.2f %s\n'%(hmax,lmax,qemax,qimax,flag))
    for value in ref_dict.values():
        h1,h2,h3,h4,h5 = value[0]
        f.write('%d %d %d %d %d %d %8.6f %8.6f\n'%(h1,h2,h3,h4,h5,value[1],value[2],value[3]))
    f.close()
    print('generation of reflection list: completed')
    return 0

In [20]:
filename = 'reflection_list.ref'

In [21]:
reflection_list(filename,hmax,lmax,qemax,qimax,a_max,c_max,flag)

generation of reflection list: completed


0